In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from random import *
from matplotlib import pyplot as plt
import numpy as np
import GPy
from datetime import datetime

# change here
model_class_names = [3]

# randomly sample demo_light_num rows to model our function
demo_light_percentage = 0.1;

# sample sample_time_index_num time-indexes/columns from the data (1024 columns totally)
sample_time_index_num = 10;

lightcurve_train_set_path = "./StarLightCurves/StarLightCurves_TRAIN"

In [2]:
# sample_time_index_num time indexes from 1024 indexes
times_indexes = np.asarray(sorted(sample(list(range(0, 1024)), sample_time_index_num)))
# times_indexes = np.arange(0,1024, 1024/sample_time_index_num, dtype=int);
print("We choose the following ",sample_time_index_num, "time indexes from 1024 ones\n", times_indexes)

We choose the following  10 time indexes from 1024 ones
 [ 74 142 256 403 441 473 489 560 641 832]


In [3]:
# class name {1,2,3}
class_names = np.loadtxt(lightcurve_train_set_path, delimiter=',', usecols=[0])

# light curve data
light_curve = np.loadtxt(lightcurve_train_set_path, delimiter=',', usecols=range(1, 1025))

# normalize magnitude
light_curve -= light_curve.mean(1)[:,np.newaxis]
light_curve /= light_curve.std(1)[:,np.newaxis]

In [4]:

indices = [i for i,cn in enumerate(class_names) if cn in model_class_names]
print("model_class_names=",model_class_names," totally has ", len(indices), " rows");

demo_light_num = int(demo_light_percentage * len(indices))
indices = sorted(sample(indices, demo_light_num))
# indices = indices[0:demo_light_num]
print("We select the following ",len(indices)," rows\n",indices)

kern_class = GPy.kern.Matern32(input_dim=1, variance=1.5, lengthscale=2.5, active_dims=[0], name='class')
kern_observation = GPy.kern.Matern32(input_dim=1, variance=.1, lengthscale=2.5, active_dims=[0], name='observation')
k_hierarchy = GPy.kern.Hierarchical(kernels=[kern_class, kern_observation])
print(k_hierarchy)

time_index_stack = np.tile(times_indexes.reshape(-1,1),(demo_light_num, 1)) ;

observation_stack = []
for r in range(1, demo_light_num + 1):
    observation_stack.append(np.ones(sample_time_index_num) * r);

observation_stack = np.asarray(observation_stack).reshape(-1,1);

filtered_light_curve = light_curve[indices]
filtered_light_curve = filtered_light_curve[:,times_indexes]

X = np.hstack((time_index_stack, observation_stack))
X = np.vstack(X)
Y = np.vstack(filtered_light_curve.reshape(-1,1))

print(X.shape)
print(Y.shape)

X_file_path = "./StarLight_files/X_files/X_" + str(model_class_names)  + ".npy"
Y_file_path = "./StarLight_files/Y_files/Y_" + str(model_class_names)  + ".npy"

np.save(X_file_path, X)
np.save(Y_file_path, Y)

m = GPy.models.GPRegression(X=X, Y=Y, kernel=k_hierarchy)
# m.optimize_restarts(num_restarts=3);
m.optimize('bfgs', messages=1)
print(m)

save_file_path = "./StarLight_files/model_save_files/model_save" + str(model_class_names) + ".npy"
np.save(save_file_path, m.param_array)


model_class_names= [3]  totally has  573  rows
We select the following  57  rows
 [6, 80, 96, 97, 117, 122, 192, 193, 203, 205, 220, 222, 264, 279, 320, 327, 378, 395, 436, 443, 463, 487, 512, 516, 539, 551, 584, 614, 630, 639, 641, 645, 652, 653, 707, 712, 717, 731, 763, 823, 841, 852, 858, 859, 862, 863, 870, 897, 918, 930, 952, 962, 971, 974, 986, 987, 991]
  hierarchy.               |  value  |  constraints  |  priors
  class.variance           |    1.5  |      +ve      |        
  class.lengthscale        |    2.5  |      +ve      |        
  observation.variance     |    0.1  |      +ve      |        
  observation.lengthscale  |    2.5  |      +ve      |        
(570, 2)
(570, 1)
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    02s28  0017   5.531333e+02   2.098370e-01 
    06s56  0046   5.531316e+02   1.185953e-01 
    08s72  0063   5.531315e+02   8.420080e-02 
Runtime:     08s72
Optimization status: Converged


Name : GP regressio